**Import required libraries and scripts**

In [1]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.consensus_methods import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *

software = '/home/tony/DockM8/software'
protein_file = '/home/tony/DockM8/wocondock_main/2o1x_A_apo_protoss.pdb'
ref_file = '/home/tony/DockM8/wocondock_main/2o1x_A_lig_protoss.sdf'
docking_library = '/home/tony/DockM8/wocondock_main/Selection_of_FCHGroup_LeadLike.sdf'
docking_programs = ['GNINA', 'SMINA', 'PLANTS', 'QVINAW', 'QVINA2']
clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS']
rescoring_functions = ['gnina', 'AD4', 'chemplp', 'rfscorevs', 'LinF9', 'RTMScore', 'SCORCH', 'vinardo', 'plecscore', 'nnscore']
id_column = 'ID'
n_poses = 10
exhaustiveness = 8
protonation = 'pkasolver'
# ncpus = int(os.cpu_count()-2)
ncpus = 7
pocket = 'reference'
#Create a temporary folder for all further calculations
w_dir = os.path.dirname(protein_file)
print('The working directory has been set to:', w_dir)
create_temp_folder(w_dir+'/temp')
if pocket == 'reference':
    pocket_definition = get_pocket(ref_file, protein_file, 8)
    print(pocket_definition)
elif pocket == 'dogsitescorer':
    pocket_definition = binding_site_coordinates_dogsitescorer(protein_file, w_dir, method='volume')

[18:11:56] Initializing Normalizer
/home/tony/.conda/envs/dockm8/lib/python3.8/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
/home/tony/.conda/envs/dockm8/lib/python3.8/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")
[TRJ.py:171 - <module>()] netCDF4 is not available. Writing AMBER ncdf files will be slow.
/home/tony/.conda/envs/dockm8/lib/python3.8/site-packages/MDAnalysis/coordinates/TRJ.py:1209: DeprecationWarning: Please use `netcdf_file` from the `scipy.io` namespace, the `scipy.io.netcdf` namespace is deprecated.
  class NCDFPicklable(scipy.io.

The working directory has been set to: /home/tony/DockM8/wocondock_main

[2023-Jun-06 18:12:00]: Extracting pocket from /home/tony/DockM8/wocondock_main/2o1x_A_apo_protoss.pdb using /home/tony/DockM8/wocondock_main/2o1x_A_lig_protoss.sdf as reference ligand

[2023-Jun-06 18:12:10]: Finished extracting pocket from /home/tony/DockM8/wocondock_main/2o1x_A_apo_protoss.pdb using /home/tony/DockM8/wocondock_main/2o1x_A_lig_protoss.sdf as reference ligand
{'center': [16.7, -2.69, 17.61], 'size': [16.0, 16.0, 16.0]}


In [ ]:
# if os.path.isfile(protein_file.replace('.pdb', '_pocket.pdb')) == False:


In [ ]:
if os.path.isfile(w_dir+'/temp/final_library.sdf') == False:
    prepare_library(docking_library, id_column, software, protonation, ncpus)

In [ ]:
docking(w_dir, protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses, ncpus, pocket_definition)


In [ ]:
print('Loading all poses SDF file...')
tic = time.perf_counter()
all_poses = PandasTools.LoadSDF(w_dir+'/temp/allposes.sdf', idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
toc = time.perf_counter()
print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')

In [ ]:
for metric in clustering_metrics:
        if os.path.isfile(w_dir+f'/temp/clustering/{metric}_clustered.sdf') == False:
            cluster_pebble(metric, 'KMedoids', w_dir, protein_file, all_poses, ncpus)

In [2]:
for metric in ['RMSD']:
        rescore_all(w_dir, protein_file, pocket_definition, software, w_dir+f'/temp/clustering/{metric}_clustered.sdf', rescoring_functions, ncpus)



[2023-Jun-06 18:12:10]: Skipping gnina rescoring...

[2023-Jun-06 18:12:10]: Skipping AD4 rescoring...

[2023-Jun-06 18:12:10]: Skipping chemplp rescoring...

[2023-Jun-06 18:12:10]: Skipping rfscorevs rescoring...
Splitting SDF file RMSD_clustered.sdf ...


Splitting files: 100%|██████████| 12/12 [00:00<00:00, 476.68it/s]


Split docking library into 12 files each containing 3 compounds


Rescoring with LinF9: 100%|██████████| 12/12 [00:04<00:00,  2.44file/s]



[2023-Jun-06 18:12:16]: Rescoring with LinF9 complete in 5.1771!

[2023-Jun-06 18:12:16]: Skipping RTMScore rescoring...

[2023-Jun-06 18:12:16]: Skipping SCORCH rescoring...

[2023-Jun-06 18:12:16]: Skipping vinardo rescoring...

[2023-Jun-06 18:12:16]: Skipping plecscore rescoring...

[2023-Jun-06 18:12:16]: Skipping nnscore rescoring...

[2023-Jun-06 18:12:16]: Combining all score for /home/tony/DockM8/wocondock_main/temp/rescoring_RMSD_clustered
                Pose ID    PLECnn
0   FCG17585042_GNINA_5  4.420794
1    FCG1390566_GNINA_5  2.952029
2  FCG16425532_GNINA_10  4.911706
3   FCG16600623_GNINA_2  5.092506
4   FCG18066182_GNINA_1  4.514408


Combining scores: 100%|██████████| 9/9 [00:00<00:00, 536.37files/s]

                Pose ID    PLECnn   SCORCH       AD4  CHEMPLP   NNscore   
0   FCG17585042_GNINA_5  4.420794  0.00149 -32.44570 -15.8440  6.627476  \
1    FCG1390566_GNINA_5  2.952029  0.02408 -28.27369 -49.7480  5.694674   
2  FCG16425532_GNINA_10  4.911706  0.00021 -46.72932 -64.1537  7.298304   
3   FCG16600623_GNINA_2  5.092506  0.00199 -44.28569 -18.2377  7.089444   
4   FCG18066182_GNINA_1  4.514408  0.00142 -41.07597 -47.7745  7.087881   

    RTMScore    LinF9  Vinardo  GNINA_Affinity  CNN-Score  CNN-Affinity   
0   8.449504 -5.73788 -4.49190        -6.10025   0.463739      5.882300  \
1  10.265566 -4.83062 -4.00009        -4.93958   0.744352      5.851086   
2  13.239522 -8.54065 -7.48047        -8.32611   0.636539      6.895614   
3  17.035928 -7.90278 -5.64959        -6.57347   0.739295      6.962545   
4  11.444714 -8.39903 -6.08973        -8.13380   0.853096      7.174075   

   RFScoreVS  
0   5.971980  
1   5.973758  
2   5.984559  
3   6.003579  
4   5.988194  
        

In [ ]:
calculate_EF_single_functions(w_dir, docking_library, clustering_metrics)

In [ ]:
apply_consensus_methods_combinations(w_dir, docking_library, clustering_metrics)

In [ ]:
apply_consensus_methods(w_dir, 'bestpose', 'method6', ['gnina', 'RTMScore'])